In [1]:
# Necessary Packages
import gurobipy as gp
from gurobipy import GRB

import numpy as np
import pandas as pd
import math

In [2]:
# Bring in all data
attenuators = pd.read_csv('attenuators.csv')
brakes = pd.read_csv('brakes.csv')
cabins = pd.read_csv('cabins.csv')
materials = pd.read_csv('materials.csv')
engines = pd.read_csv('motors.csv')
suspensions = pd.read_csv('suspension.csv')
tires = pd.read_csv('tires.csv')
wings = pd.read_csv('wings.csv')

# Printing all data frames for reference
print('ATTENUATORS')
print(attenuators.to_string())
print()
print()
print('BRAKES')
print(brakes.to_string())
print()
print()
print('CABINS')
print(cabins.to_string())
print()
print()
print('MATERIALS')
print(materials.to_string())
print()
print()
print('ENGINES')
print(engines.to_string())
print()
print()
print('SUSPENSION')
print(suspensions.to_string())
print()
print()
print('TIRES')
print(tires.to_string())
print()
print()
print('WINGS')
print(wings.to_string())
print()
print()

ATTENUATORS
    Unnamed: 0  length  height  width  length*heigth*width (Ordering Impact Attenuator)
0            0    0.20     0.5    0.5                                            0.0500
1            1    0.20     0.5    0.5                                            0.0500
2            2    0.20     0.5    0.5                                            0.0500
3            3    0.20     0.5    0.5                                            0.0500
4            4    0.20     0.5    0.5                                            0.0500
5            5    0.20     0.5    0.5                                            0.0500
6            6    0.20     0.5    0.5                                            0.0500
7            7    0.20     0.5    0.5                                            0.0500
8            8    0.20     0.5    0.5                                            0.0500
9            9    0.20     0.5    0.5                                            0.0500
10          10    0.

In [3]:
# Objective Mins and Maxes
#
# Order: 
# 0 - Mass
# 1 - Center of Gravity
# 2 - Drag
# 3 - Downforce
# 4 - Acceleration
# 5 - Crash Force
# 6 - Impact Attenuator Volume
# 7 - Cornering Velocity
# 8 - Braking Distance
# 9 - Suspension Acceleration
# 10 - Pitch Moment

objectives_min = [95.4413,
                  0.1159,
                  4.8283,
                  0.006299,
                  0.0,
                  1636425.0,
                  0.004049,
                  0.01908,
                  5.9088,
                  9.81,
                  0.03386]

objectives_max = [5593.26,
                   0.9996,
                   633.95,
                   3211.16,
                   4.35,
                   672530217,
                   0.1579,
                   15.0652,
                   553.92,
                   18.79,
                   9850.87]

In [4]:
# Scratch Work
print(attenuators.iloc[0]['length'])
print()

print(len(attenuators))
print()

print(tires.iloc[0]['mass'])

print()

test_value = 0
for rt in range(len(tires)):
    test_value += tires.iloc[rt]['radius']
print(test_value)
print(len(tires))
test_value = 2 * (test_value / len(tires))
print(test_value)

0.2

64

3.636

1.7132299999999998
7
0.48949428571428566


In [5]:
m = gp.Model()

# Declare Variables
# Rear Wing
x_rw = m.addVars([
    (rw)
    for rw in range(len(wings))
    ], vtype = GRB.BINARY, name = 'Rear Wing')

# Front Wing
x_fw = m.addVars([
    (fw)
    for fw in range(len(wings))
    ], vtype = GRB.BINARY, name = 'Front Wing')

# Side Wing
x_sw = m.addVars([
    (sw)
    for sw in range(len(wings))
    ], vtype = GRB.BINARY, name = 'Side Wing')

# Front Tires
x_ft = m.addVars([
    (ft)
    for ft in range(len(tires))
    ], vtype = GRB.BINARY, name = 'Front Tires')

# Rear Tires
x_rt = m.addVars([
    (rt)
    for rt in range(len(tires))
    ], vtype = GRB.BINARY, name = 'Rear Tires')

# Cabin
x_c = m.addVars([
    (c)
    for c in range(len(cabins))
    ], vtype = GRB.BINARY, name = 'Cabin')

# Engine
x_e = m.addVars([
    (e)
    for e in range(len(engines))
    ], vtype = GRB.BINARY, name = 'Engine')

# Attenuator
x_ia = m.addVars([
    (ia)
    for ia in range(len(attenuators))
    ], vtype = GRB.BINARY, name = 'Impact Attenuator')

# Brakes
x_brk = m.addVars([
    (brk)
    for brk in range(len(brakes))
    ], vtype = GRB.BINARY, name = 'Brakes')

# Suspension
x_sp = m.addVars([
    (sp)
    for sp in range(len(suspensions))
    ], vtype = GRB.BINARY, name = 'Suspension')

# Rear Wing Material
x_rw_mat = m.addVars([
    (rw_mat)
    for rw_mat in range(len(materials))
    ], vtype = GRB.BINARY, name = 'Rear Wing Material')

# Front Wing Material
x_fw_mat = m.addVars([
    (fw_mat)
    for fw_mat in range(len(materials))
    ], vtype = GRB.BINARY, name = 'Front Wing Material')

# Side Wing Material
x_sw_mat = m.addVars([
    (sw_mat)
    for sw_mat in range(len(materials))
    ], vtype = GRB.BINARY, name = 'Side Wing Material')

# Cabin Material
x_c_mat = m.addVars([
    (c_mat)
    for c_mat in range(len(materials))
    ], vtype = GRB.BINARY, name = 'Cabin Material')

# Attenuator Material
x_ia_mat = m.addVars([
    (ia_mat)
    for ia_mat in range(len(materials))
    ], vtype = GRB.BINARY, name = 'Attenuator Material')

# Ansilary Things
weights = [0.14, 0.01, 0.20, 0.30, 0.10, 0.01, 0.01, 0.10, 0.10, 0.02, 0.01] # Weights of each term in objective function (Scenario 1)
cost_weight = 0.2
for obj in range(len(weights)):
    weights[obj] = weights[obj] * cost_weight

objectives_order = ['m', 'CG_y', 'F_d', 'F_R', 'a_car', 'F_crash', 'V_ia', 'V_cor', 'D_brk', 'a_sp', 'M_pitch', 'C_total']
v_car = 26.8 #m/s
omega_e = 3600 #rpm
rho_air = 1.225 #kg/m^3
r_track = 9 #m
P_brk = 1 * (10 ** 7) #Pa
tire_pressure = 0.904 # Medium pressure on pressure.csv file in Dr. McComb's Github
C = 0.005 + ((1 / tire_pressure) * (0.01 + (0.0095 * (((v_car * 3.6) / (100)) ** 2))))
g = 9.81 #m/s^2
y_par = 0.05 #m
y_dot_par = 0.025 #m/s
c_brk = 0.5 # Coefficient of Brake Friction
l_f = 0.5 # Pitch Moment Parameter
l_c = sum(cabins.iloc[c]['length'] for c in range(len(cabins))) / len(cabins)

# Declare Objective Equivalence Variables (y)
y = m.addVars([
    (obj)
    for obj in objectives_order
], vtype = GRB.CONTINUOUS, name = 'Objective Equivalence')

# Declare Attenuator and Material (x_{ia} and x_{mat_ia}) equivalence Variable
w0 = m.addVars([
    (x_ia_eq, x_mat_ia_eq)
    for x_ia_eq in range(len(attenuators))
    for x_mat_ia_eq in range(len(materials))
], vtype = GRB.BINARY, name = 'Equivalence Variable: x_ia and x_mat_ia')

# Declare Square of V_cor Equivalence Variable
w1 = m.addVar(1, vtype = GRB.CONTINUOUS, name = 'Square of V_cor Equivalence Variable')

# Declare Braking Distance and Suspension Choice Equivalence Variable
w2 = m.addVars([
    (index0, sp)
    for index0 in range(1)
    for sp in range(len(suspensions))
])

# Declare Part Mass Equivalence Variables
part_mass_list = ['m_rw', 'm_fw', 'm_sw', 'm_c', 'm_ia']
z = m.addVars([
    (z_vars)
    for z_vars in part_mass_list
])

# Declare Square Root of Total Mass Equivalence Variable
w3 = m.addVar(1, vtype = GRB.CONTINUOUS, name = 'Square Root of Total Mass Equivalence Variable')

# Formulate Objectives

# Mass Objectives
rear_wing_mass = gp.quicksum(wings.iloc[rw]['length'] * wings.iloc[rw]['width'] * wings.iloc[rw]['height']
                            * x_rw[rw] for rw in range(len(wings))) * gp.quicksum(materials.iloc[rw_mat]['q'] * x_rw_mat[rw_mat] for rw_mat in range(len(materials)))

front_wing_mass = gp.quicksum(wings.iloc[fw]['length'] * wings.iloc[fw]['width'] * wings.iloc[fw]['height']
                             * x_fw[fw] for fw in range(len(wings))) * gp.quicksum(materials.iloc[fw_mat]['q'] * x_fw_mat[fw_mat] for fw_mat in range(len(materials)))

side_wing_mass = gp.quicksum(wings.iloc[sw]['length']  * wings.iloc[sw]['width'] * wings.iloc[sw]['height'] 
                             * x_sw[sw] for sw in range(len(wings))) * gp.quicksum(materials.iloc[sw_mat]['q'] * x_sw_mat[sw_mat] for sw_mat in range(len(materials)))

front_tire_mass = gp.quicksum(tires.iloc[ft]['mass'] * x_ft[ft] for ft in range(len(tires)))

rear_tire_mass = gp.quicksum(tires.iloc[rt]['mass'] * x_rt[rt] for rt in range(len(tires)))

cabin_mass = 2 * gp.quicksum(cabins.iloc[c]['thickness'] * (cabins.iloc[c]['length'] * cabins.iloc[c]['width'] + 
                                                       cabins.iloc[c]['length'] * cabins.iloc[c]['height'] + 
                                                       cabins.iloc[c]['width'] * cabins.iloc[c]['height']) 
                         * x_c[c] for c in range(len(cabins))) * gp.quicksum(materials.iloc[c_mat]['q'] * 
                                                                            x_c_mat[c_mat] for c_mat in range(len(materials)))

engine_mass = gp.quicksum(engines.iloc[e]['Mass'] * x_e[e] for e in range(len(engines)))

attenuator_mass = (gp.quicksum(attenuators.iloc[ia]['length'] * attenuators.iloc[ia]['width'] * 
                             attenuators.iloc[ia]['height'] * x_ia[ia] for ia in range(len(attenuators))) * 
                             gp.quicksum(materials.iloc[ia_mat]['q'] * x_ia_mat[ia_mat] for ia_mat in range(len(materials))))

brakes_mass = gp.quicksum(brakes.iloc[brk]['lbrk'] * brakes.iloc[brk]['wbrk'] * brakes.iloc[brk]['hbrk'] * 
                             brakes.iloc[brk]['qbrk'] * x_brk[brk] for brk in range(len(brakes)))

suspensions_mass = gp.quicksum((suspensions.iloc[sp]['mrsp'] + suspensions.iloc[sp]['mfsp']) * 
                               x_sp[sp] for sp in range(len(suspensions)))

total_mass = (rear_wing_mass + front_wing_mass + 2 * side_wing_mass + 
              2 * front_tire_mass + 2 * rear_tire_mass + cabin_mass + engine_mass 
              + attenuator_mass + 4 * brakes_mass + 2 * suspensions_mass)


# Center of Gravity Objectives
rear_wing_position_y = (2 * (sum(tires.iloc[rt]['radius'] for rt in range(len(tires))) / len(tires)) +
                       (0.5 * (sum(wings.iloc[rw]['height'] for rw in range(len(wings))) / len(wings))))

front_wing_position_y = (0.75 * (sum(tires.iloc[ft]['radius'] for ft in range(len(tires))) / len(tires)))

side_wing_position_y = (sum(tires.iloc[ft]['radius'] for ft in range(len(tires))) / len(tires))

rear_tire_position_y = (sum(tires.iloc[rt]['radius'] for rt in range(len(tires))) / len(tires))

front_tire_position_y = (sum(tires.iloc[ft]['radius'] for ft in range(len(tires))) / len(tires))

cabin_position_y = ((sum(tires.iloc[rt]['radius'] for rt in range(len(tires))) / len(tires)) + 
                   (sum(cabins.iloc[c]['height'] for c in range(len(cabins))) / len(cabins)))

engine_position_y = ((sum(tires.iloc[rt]['radius'] for rt in range(len(tires))) / len(tires)) + 
                    (sum(engines.iloc[e]['Height'] for e in range(len(engines))) / len(engines)))

attenuator_position_y = ((sum(tires.iloc[ft]['radius'] for ft in range(len(tires))) / len(tires)) + 0.25)

brakes_position_y = (1.5 * (sum(tires.iloc[ft]['radius'] for ft in range(len(tires))) / len(tires)))

rear_suspension_position_y = (sum(tires.iloc[rt]['radius'] for rt in range(len(tires))) / len(tires))

front_suspension_position_y = (sum(tires.iloc[ft]['radius'] for ft in range(len(tires))) / len(tires))

# Constraint on Center of Gravity Objective
m.addConstr(y['CG_y'] * y['m'] == ((rear_wing_mass * rear_wing_position_y) + (front_wing_mass * front_wing_position_y) + 
                                   (2 * side_wing_mass * side_wing_position_y) + (2 * front_tire_mass * front_tire_position_y) + 
                                   (2 * rear_tire_mass * rear_tire_position_y) + (cabin_mass * cabin_position_y) + 
                                   (engine_mass * engine_position_y) + (attenuator_mass * attenuator_position_y) + 
                                   (4 * brakes_mass * brakes_position_y) + (suspensions_mass * (0.5 * rear_suspension_position_y * 
                                                                                                front_suspension_position_y))))


# Downforce Objectives

rear_wing_downforce = gp.quicksum((((wings.iloc[rw]['angle of attack'] ** 2) * (wings.iloc[rw]['width'] ** 2) * 
                                   (v_car ** 2) * wings.iloc[rw]['height'] * rho_air * math.pi * math.cos(wings.iloc[rw]['angle of attack'])) / 
                                   ((wings.iloc[rw]['width'] * math.cos(wings.iloc[rw]['angle of attack'])) + (2 * wings.iloc[rw]['length']))) * x_rw[rw] for rw in range(len(wings)))

front_wing_downforce = gp.quicksum((((wings.iloc[fw]['angle of attack'] ** 2) * (wings.iloc[fw]['width'] ** 2) * 
                                   (v_car ** 2) * wings.iloc[fw]['height'] * rho_air * math.pi * math.cos(wings.iloc[fw]['angle of attack'])) / 
                                   ((wings.iloc[fw]['width'] * math.cos(wings.iloc[fw]['angle of attack'])) + (2 * wings.iloc[fw]['length']))) * x_fw[fw] for fw in range(len(wings)))

side_wing_downforce = gp.quicksum((((wings.iloc[sw]['angle of attack'] ** 2) * (wings.iloc[sw]['width'] ** 2) * 
                                   (v_car ** 2) * wings.iloc[sw]['height'] * rho_air * math.pi * math.cos(wings.iloc[sw]['angle of attack'])) / 
                                   ((wings.iloc[sw]['width'] * math.cos(wings.iloc[sw]['angle of attack'])) + (2 * wings.iloc[sw]['length']))) * x_sw[sw] for sw in range(len(wings)))

total_downforce = rear_wing_downforce + front_wing_downforce + (2 * side_wing_downforce)


# Drag Objectives

rear_wing_drag = (gp.quicksum(((2 * (wings.iloc[rw]['width'] ** 2) * (wings.iloc[rw]['angle of attack'] ** 2) * 
                               (v_car ** 2) * wings.iloc[rw]['height'] * rho_air * math.pi * math.cos(wings.iloc[rw]['angle of attack'])) / 
                               (wings.iloc[rw]['length'] * ((((wings.iloc[rw]['width'] * math.cos(wings.iloc[rw]['angle of attack'])) / (wings.iloc[rw]['length'])) + 2) ** 2))) * x_rw[rw] for rw in range(len(wings))))

front_wing_drag = (gp.quicksum(((2 * (wings.iloc[fw]['width'] ** 2) * (wings.iloc[fw]['angle of attack'] ** 2) * 
                               (v_car ** 2) * wings.iloc[fw]['height'] * rho_air * math.pi * math.cos(wings.iloc[fw]['angle of attack'])) / 
                               (wings.iloc[fw]['length'] * ((((wings.iloc[fw]['width'] * math.cos(wings.iloc[fw]['angle of attack'])) / (wings.iloc[fw]['length'])) + 2) ** 2))) * x_fw[fw] for fw in range(len(wings))))

side_wing_drag = (gp.quicksum(((2 * (wings.iloc[sw]['width'] ** 2) * (wings.iloc[sw]['angle of attack'] ** 2) * 
                               (v_car ** 2) * wings.iloc[sw]['height'] * rho_air * math.pi * math.cos(wings.iloc[sw]['angle of attack'])) / 
                               (wings.iloc[sw]['length'] * ((((wings.iloc[sw]['width'] * math.cos(wings.iloc[sw]['angle of attack'])) / (wings.iloc[sw]['length'])) + 2) ** 2))) * x_sw[sw] for sw in range(len(wings))))


cabin_drag = (gp.quicksum(0.02 * rho_air * (v_car ** 2) * cabins.iloc[c]['width'] * cabins.iloc[c]['height'] * x_c[c] for c in range(len(cabins))))

total_drag = rear_wing_drag + front_wing_drag + (2 * side_wing_drag) + cabin_drag


# Acceleration Objective Constraint

m.addConstr(y['a_car'] * y['m'] == (gp.quicksum((((engines.iloc[e]['Torque'] * (3600 * 2 * math.pi / 60)) / (v_car)) - (y['F_R'] + (C * y['m'] * g))) * x_e[e] for e in range(len(engines)))))


# Crash Force Objective

m.addConstr((y['F_crash'] == gp.quicksum((math.sqrt(((v_car ** 2) * attenuators.iloc[ia]['width'] * 
                                                    attenuators.iloc[ia]['height'] * materials.iloc[ia_mat]['E']) / 
                                                    (2 * attenuators.iloc[ia]['length']))) * 
                                         w3 * w0[ia, ia_mat] for ia in range(len(attenuators)) for ia_mat in range(len(materials)))))

# Impact Attenuator Volume Objective

attenuator_volume = (gp.quicksum(attenuators.iloc[ia]['length'] * attenuators.iloc[ia]['width'] * 
                                attenuators.iloc[ia]['height'] * x_ia[ia] for ia in range(len(attenuators))))


# Cornering Velocity Objective

m.addConstr((w1 * y['m'] == (gp.quicksum((y['F_d'] + (y['m'] * g) - (2 * (suspensions.iloc[sp]['kfsp'] * y_par + suspensions.iloc[sp]['cfsp'] * y_dot_par)) 
                             - (2 * (suspensions.iloc[sp]['krsp'] * y_par + suspensions.iloc[sp]['crsp'] * y_dot_par))) * C * r_track * x_sp[sp] for sp in range(len(suspensions))))))

# Braking Distance Objective

m.addConstr((gp.quicksum(w2[0,sp] * (sum(tires.iloc[t]['radius'] for t in range(len(tires))) / len(tires)) *
                        C * ((y['m'] * g) + y['F_d'] - (2 * (suspensions.iloc[sp]['kfsp'] * y_par + suspensions.iloc[sp]['cfsp'] * y_dot_par)) - 
                            (2 * (suspensions.iloc[sp]['krsp'] * y_par + suspensions.iloc[sp]['crsp'] * y_dot_par))) for sp in range(len(suspensions))) == ((
            ((v_car ** 2) * (sum(tires.iloc[t]['radius'] for t in range(len(tires))) / len(tires)) * y['m'])) - 
            (16 * y['D_brk'] * gp.quicksum(C * P_brk * brakes.iloc[brk]['lbrk'] * brakes.iloc[brk]['wbrk'] * brakes.iloc[brk]['rbrk'] *
                                         c_brk * x_brk[brk] for brk in range(len(brakes)))))))


# Suspension Acceleration Objective

m.addConstr((y['a_sp'] * y['m'] == (gp.quicksum(((-2 * (suspensions.iloc[sp]['kfsp'] * y_par + suspensions.iloc[sp]['cfsp'] * y_dot_par)) + 
                                               (2 * (suspensions.iloc[sp]['krsp'] * y_par + suspensions.iloc[sp]['crsp'] * y_dot_par)) + 
                                               (y['m'] * g) + (y['F_d'])) * x_sp[sp] for sp in range(len(suspensions))))))


# Pitch Moment Objective

suspension_pitch_moment = gp.quicksum(((2 * (suspensions.iloc[sp]['kfsp'] * y_par + suspensions.iloc[sp]['cfsp'] * y_dot_par)) + 
                                      (2 * (suspensions.iloc[sp]['krsp'] * y_par + suspensions.iloc[sp]['crsp'] * y_dot_par))) * x_sp[sp] for sp in range(len(suspensions)))


rear_wing_pitch_moment = gp.quicksum(rear_wing_downforce * (l_c - wings.iloc[rw]['length']) * x_rw[rw] for rw in range(len(wings)))

front_wing_pitch_moment = gp.quicksum(front_wing_downforce * (l_c - wings.iloc[fw]['length']) * x_fw[fw] for fw in range(len(wings)))

side_wing_pitch_moment = gp.quicksum(side_wing_downforce * (l_c - wings.iloc[sw]['length']) * x_sw[sw] for sw in range(len(wings)))

pitch_moment_total = suspension_pitch_moment + rear_wing_pitch_moment - front_wing_pitch_moment - (2 * side_wing_pitch_moment)

# Cost Objective

rw_cost = z['m_rw'] * gp.quicksum(materials.iloc[rw_mat]['cost_per_kilogram'] * x_rw_mat[rw_mat] for rw_mat in range(len(materials)))

fw_cost = z['m_fw'] * gp.quicksum(materials.iloc[fw_mat]['cost_per_kilogram'] * x_fw_mat[fw_mat] for fw_mat in range(len(materials)))

sw_cost = z['m_sw'] * gp.quicksum(materials.iloc[sw_mat]['cost_per_kilogram'] * x_sw_mat[sw_mat] for sw_mat in range(len(materials)))

ft_cost = gp.quicksum(tires.iloc[ft]['cost'] * x_ft[ft] for ft in range(len(tires))) * ((1 + (tire_pressure - 0.758)) / 2)

rt_cost = gp.quicksum(tires.iloc[rt]['cost'] * x_rt[rt] for rt in range(len(tires))) * ((1 + (tire_pressure - 0.758)) / 2)

engine_cost = gp.quicksum(engines.iloc[e]['Cost'] * x_e[e] for e in range(len(engines)))

cabin_cost = z['m_c'] * gp.quicksum(materials.iloc[c_mat]['cost_per_kilogram'] * x_c_mat[c_mat] for c_mat in range(len(materials)))

ia_cost = z['m_ia'] * gp.quicksum(materials.iloc[ia_mat]['cost_per_kilogram'] * x_ia_mat[ia_mat] for ia_mat in range(len(materials)))

brake_cost = brakes_mass * 25

suspension_cost = 0

total_cost = (rw_cost + fw_cost + (2 * sw_cost) + (2 * ft_cost) + (2 * rt_cost) + 
              engine_cost + cabin_cost + ia_cost + (4 * brake_cost) + suspension_cost)


# Total Objective
total_objective = ((weights[0] * ((y['m'] - objectives_min[0]) / (objectives_max[0] - objectives_min[0]))) + 
                   (weights[1] * ((y['CG_y'] - objectives_min[1]) / (objectives_max[1] - objectives_min[1]))) + 
                   (weights[2] * ((y['F_R'] - objectives_min[2]) / (objectives_max[2] - objectives_min[2]))) + 
                   (weights[3] * ((-y['F_d'] + objectives_max[3]) / (-objectives_min[3] + objectives_max[3]))) +
                   (weights[4] * ((-y['a_car'] + objectives_max[4]) / (-objectives_min[4] + objectives_max[4]))) + 
                   (weights[5] * ((y['F_crash'] - objectives_min[5]) / (objectives_max[5] - objectives_min[5]))) +
                   (weights[6] * ((y['V_ia'] - objectives_min[6]) / (objectives_max[6] - objectives_min[6]))) + 
                   (weights[7] * ((-y['V_cor'] + objectives_max[7]) / (-objectives_min[7] + objectives_max[7]))) + 
                   (weights[8] * ((y['D_brk'] - objectives_min[8]) / (objectives_max[8] - objectives_min[8]))) + 
                   (weights[9] * ((y['a_sp'] - objectives_min[9]) / (objectives_max[9] - objectives_min[9]))) + 
                   (weights[10] * ((y['M_pitch'] - objectives_min[10]) / (objectives_max[10] - objectives_min[10]))) + 
                   (cost_weight * ((y['C_total'] - 1500) / (200000 - 1500))))

m.setObjective(total_objective, GRB.MINIMIZE)

# Constraints on Car Parts
m.addConstr((gp.quicksum(x_rw[rw] for rw in range(len(wings))) == 1), name = 'One Rear Wing')
m.addConstr((gp.quicksum(x_fw[fw] for fw in range(len(wings))) == 1), name = 'One Front Wing')
m.addConstr((gp.quicksum(x_sw[sw] for sw in range(len(wings))) == 1), name = 'One Side Wing')
m.addConstr((gp.quicksum(x_ft[ft] for ft in range(len(tires))) == 1), name = 'One Front Tire')
m.addConstr((gp.quicksum(x_rt[rt] for rt in range(len(tires))) == 1), name = 'One Rear Tire')
m.addConstr((gp.quicksum(x_c[c] for c in range(len(cabins))) == 1), name = 'One Cabin')
m.addConstr((gp.quicksum(x_e[e] for e in range(len(engines))) == 1), name = 'One Engine')
m.addConstr((gp.quicksum(x_ia[ia] for ia in range(len(attenuators))) == 1), name = 'One Attenuator')
m.addConstr((gp.quicksum(x_brk[brk] for brk in range(len(brakes))) == 1), name = 'One Brakes')
m.addConstr((gp.quicksum(x_sp[sp] for sp in range(len(suspensions))) == 1), name = 'One Suspension')
m.addConstr((gp.quicksum(x_rw_mat[rw_mat] for rw_mat in range(len(materials))) == 1), name = 'One RW MAterial')
m.addConstr((gp.quicksum(x_fw_mat[fw_mat] for fw_mat in range(len(materials))) == 1), name = 'One FW Material')
m.addConstr((gp.quicksum(x_sw_mat[sw_mat] for sw_mat in range(len(materials))) == 1), name = 'One SW Material')
m.addConstr((gp.quicksum(x_c_mat[c_mat] for c_mat in range(len(materials))) == 1), name = 'One Cabin Material')
m.addConstr((gp.quicksum(x_ia_mat[ia_mat] for ia_mat in range(len(materials))) == 1), name = 'One IA Material')

# Constraints on Objective Equivalence Variables
m.addConstr(y['m'] == total_mass)
m.addConstr(y['F_d'] == total_downforce)
m.addConstr(y['F_R'] == total_drag)
m.addConstr(y['V_ia'] == attenuator_volume)
m.addConstr(y['M_pitch'] == pitch_moment_total)
m.addConstr(y['C_total'] == total_cost)

# Constraints on Part Mass Equivalence Variables
m.addConstr(z['m_rw'] == rear_wing_mass)
m.addConstr(z['m_fw'] == front_wing_mass)
m.addConstr(z['m_sw'] == side_wing_mass)
m.addConstr(z['m_c'] == cabin_mass)
m.addConstr(z['m_ia'] == attenuator_mass)

# Constraints on Equivalence Variables
m.addConstrs((w0[x_ia_eq, x_mat_ia_eq] == x_ia[x_ia_eq] * x_ia_mat[x_mat_ia_eq]
             for x_ia_eq in range(len(attenuators)) for x_mat_ia_eq in range(len(materials))), name = 'Attenuator Equivalence Variable Enforcement')

m.addConstr((w1 == y['V_cor'] * y['V_cor']), name = 'Square V_cor Equivalence Variable Enforcement')

m.addConstrs((w2[0,sp] == y['D_brk'] * x_sp[sp] for sp in range(len(suspensions))), name = 'Braking Distance and Suspension Choice Variable Equivalence Enforcement')

m.addConstr((w3 * w3 == y['m']), name = 'Enforce Mass Square Root Equivalence Variable')

# Set Non-Convex Parameter to 2
m.params.NonConvex = 2

# Parameter to enforce tighter tolerances
m.params.NumericFocus = 3

# Parameter to determine infeasibility quicker
m.params.BarHomogeneous = 1

# Only Use Primal Simpelx
m.params.Method = 1

m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-26
Set parameter NonConvex to value 2
Set parameter NumericFocus to value 3
Set parameter BarHomogeneous to value 1
Set parameter Method to value 1
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i9-9880H CPU @ 2.30GHz
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 18 rows, 1899 columns and 2382 nonzeros
Model fingerprint: 0x9f8ab5b2
Model has 853 quadratic constraints
Variable types: 24 continuous, 1875 integer (1875 binary)
Coefficient statistics:
  Matrix range     [7e-03, 5e+03]
  QMatrix range    [6e-03, 1e+07]
  QLMatrix range   [4e-05, 6e+03]
  Objective range  [3e-12, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.15s
Presolved: 242426 rows, 82502 columns, 674116 nonzeros
Presolved model has 78 SOS constraint(s)
P

In [6]:
# Print Which Rear Wing Chosen
for rw in range(len(wings)):
    if x_rw[rw].X >= 0.5:
        print('Rear Wing', rw, 'Chosen')
        
# Print Which Rear Wing Material Chosen
for rw_mat in range(len(materials)):
    if x_rw_mat[rw_mat].X >= 0.5:
        print(materials.iloc[rw_mat]['Material'], 'Chosen for Rear Wing')
        
print()

# Print Which Front Wing Chosen
for fw in range(len(wings)):
    if x_fw[fw].X >= 0.5:
        print('Front Wing', fw, 'Chosen')
        
# Print Which Rear Wing Material Chosen
for fw_mat in range(len(materials)):
    if x_fw_mat[fw_mat].X >= 0.5:
        print(materials.iloc[fw_mat]['Material'], 'Chosen for Front Wing')
        
print()

# Print Which Rear Wing Chosen
for sw in range(len(wings)):
    if x_sw[sw].X >= 0.5:
        print('Side Wing', sw, 'Chosen')
        
# Print Which Rear Wing Material Chosen
for sw_mat in range(len(materials)):
    if x_sw_mat[sw_mat].X >= 0.5:
        print(materials.iloc[sw_mat]['Material'], 'Chosen for Side Wing')
        
print()

# Print Which Front Tire Chosen
for ft in range(len(tires)):
    if x_ft[ft].X == 1:
        print('Front Tire', tires.iloc[ft]['ID'], 'Chosen')

print()

# Print Which Rear Tire Chosen
for rt in range(len(tires)):
    if x_rt[rt].X == 1:
        print('Rear Tire', tires.iloc[rt]['ID'], 'Chosen')
        
print()
        
# Print Which Cabin Chosen
for c in range(len(cabins)):
    if x_c[c].X == 1:
        print('Cabin', c, 'Chosen')
        
# Print Which Cabin Material Chosen
for c_mat in range(len(materials)):
    if x_c_mat[c_mat].X == 1:
        print(materials.iloc[c_mat]['Material'], 'Chosen for Cabin')
        
print()
        
# Print Which Engine Chosen
for e in range(len(engines)):
    if x_e[e].X == 1:
        print('Engine', engines.iloc[e]['ID'], 'Chosen')
        
print()

# Print Which Attenuator Chosen
for ia in range(len(attenuators)):
    if x_ia[ia].X == 1:
        print('Attenuator', ia, 'Chosen')
        
# Print Which Attenuator Material Chosen
for ia_mat in range(len(materials)):
    if x_ia_mat[ia_mat].X == 1:
        print(materials.iloc[ia_mat]['Material'], 'Chosen for Attenuator')
        
print()

# Print Which Brakes Chosen
for brk in range(len(brakes)):
    if x_brk[brk].X == 1:
        print(brakes.iloc[brk]['brakeID'], 'Chosen for Brakes')
        
print()
        
# Print Which Suspension Chosen
for sp in range(len(suspensions)):
    if x_sp[sp].X == 1:
        print(suspensions.iloc[sp]['id'], 'Chosen for Suspensions')
        
print()
        
# Print Total Mass
print('Total Mass:', round(y['m'].X, 4))

# Print Total Center of Gravity in y-direction
print('Center of Gravity in y:', round(y['CG_y'].X, 4))

# Print Total Drag
print('Total Drag:', round(y['F_R'].X, 4))

# Print Total Downforce
print('Total Downforce:', round(y['F_d'].X, 4))

# Print Total Acceleration
print('Total Acceleration:', round(y['a_car'].X, 4))

# Print Total Crash Force
print('Crash Force:', round(y['F_crash'].X, 4))

# Print Attenuator Volume
print('Attenuator Volume:', round(y['V_ia'].X, 4))

# Print Cornering Velocity
print('Cornering Velocity:', round(y['V_cor'].X, 4))

# Print Braking Distance
print('Braking Distance:', round(y['D_brk'].X, 4))

# Print Suspension Acceleration
print('Suspension Acceleration:', round(y['a_sp'].X, 4))

# Print Pitch Moment
print('Pitch Moment:', round(y['M_pitch'].X, 4))

# Print Cost
print('Total Cost:', round(y['C_total'].X, 4))

print()
print()

# Test Print
print('Testing w0 Variable:')
for ia in range(len(attenuators)):
    for ia_mat in range(len(materials)):
        if w0[ia,ia_mat].X >= 0.5:
            print('IA', ia, 'made of', materials.iloc[ia_mat]['Material'], 'chosen')
            

        
print()
print()


# Objective Value
print('Objective Value:', round(m.objVal, 4))

Rear Wing 133 Chosen
Polyethylene (HDPE) Chosen for Rear Wing

Front Wing 133 Chosen
Polyethylene (HDPE) Chosen for Front Wing

Side Wing 0 Chosen
Polyethylene (HDPE) Chosen for Side Wing

Front Tire T1 Chosen

Rear Tire T1 Chosen

Cabin 0 Chosen
Epoxy Chosen for Cabin

Engine M9 Chosen

Attenuator 6 Chosen
Polyethylene (HDPE) Chosen for Attenuator

B26 Chosen for Brakes

S1 Chosen for Suspensions

Total Mass: 268.1695
Center of Gravity in y: 0.4802
Total Drag: 245.965
Total Downforce: 5367.9894
Total Acceleration: 1.1441
Crash Force: 10408801.6035
Attenuator Volume: 0.05
Cornering Velocity: 2.4883
Braking Distance: 144.2788
Suspension Acceleration: 29.8272
Pitch Moment: 860.0
Total Cost: 2551.1302


Testing w0 Variable:
IA 6 made of Polyethylene (HDPE) chosen


Objective Value: 0.024
